In [20]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
from tqdm import tqdm
import concurrent.futures

from abnumber import Chain

In [8]:
# Define Sample Path, Define Humanized Path
sample_fpath = '/data/home/waitma/antibody_proj/antidiff/checkpoints/batch_one_sample_2023_11_16__22_44_23/sample_humanization_result.csv'
humanized_fpath = '/data/home/waitma/antibody_proj/antidiff/data/lab_data/humanization_pair_data_filter.csv'

In [9]:
# Get DataFrame from the fpath.
sample_df = pd.read_csv(sample_fpath)
humanized_df = pd.read_csv(humanized_fpath)

In [10]:
# Define deal out-of-order table.
def regular_order_table(out_of_order_table):
    all_table_data = []
    for table in out_of_order_table:
        table_data = []
        for row in table.find_all('tr'):
            row_data = []
            for cell in row.find_all(['th', 'td']):
                row_data.append(cell.text)
            table_data.append(row_data)
        all_table_data.append(table_data)
    return all_table_data[:2]  # only the first two will be used, all is three.

# Define extract data. Only want to know wther the sequence can be viewed as human.
def extract_human_data(regular_table):
    extracted_data = []
    for table_data in regular_table:
        table_header = table_data[0]
        human_row = [None, None, None, None]
        for row in table_data:
            if row[-1] == 'HUMAN':
                human_row = row
        extracted_data.extend(human_row)
    return extracted_data

# Define request process.
def get_predict_result(job_name, h_seq, l_seq):
    # Url path
    humab_url = 'https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab'

    data = {
        'h_sequence_score': h_seq,
        'l_sequence_score': l_seq,
        'jobname_score': job_name
    }
    reponse = requests.post(humab_url, data=data)
    result_url = reponse.url
    print(result_url)

    # Need to wait a moment.
    time.sleep(15)

    # Get the result page.
    result_response = requests.get(result_url)

    if result_response.status_code == 200:
        soup = BeautifulSoup(result_response.text, 'html.parser')
        tables = soup.find_all('table', {'class': 'table table-results'})
        # print(tables)
        
        predict_table = regular_order_table(tables)
        print(predict_table)
        extract_data = extract_human_data(predict_table)
        print(extract_data)
    else:
        print('May be url has problem or need larger sleep time.')

    sequence_list = [h_seq, l_seq]
    return extract_data + sequence_list
    

In [11]:
# Get the sample sequence from data.
# Correct dataframe name
new_columns_name = {
    'Specific': 'Specific',
    'name': 'name',
    'HSEQ': 'hseq',
    'LSEQ': 'lseq'
}

sample_df = sample_df.rename(columns=new_columns_name)
sample_human_df = sample_df[sample_df['Specific'] == 'humanization'].reset_index(drop=True)
sample_human_df.head()

,Specific,name,hseq,lseq,Unnamed: 4
0,humanization,3A3human_0,EVQLVESGGGLVQPGGSLRLSCVASGFTFSNFWMNWVRQAPGKGLE...,DIQMTQSPSPLSASVGDRANITCRASQDVSTDLGWYQQKPGKAPKV...,NaN
1,humanization,MAK195human_0,QVQLVESGGGVVQPGGSLRLSCAASGFSLTDYGVAWVRQPPGKGLE...,DIQMAQSPSSLSASVGDRVTITCRASQAVSSALTWYQQIPGKSPKL...,NaN
2,humanization,31#human_0,EVQLVQSGAEVKKPGESLKISCKASGYTFTDYWIGWVRQMPGKGLE...,DIVMTQSPDSLAVSLGERATISCKSSQSLLNTNSQKNYIAWYQQKP...,NaN
3,humanization,56#human_0,QVQLQESGPRLVEPSETLSLTCTVSGYSITSDYDWGWIRQPPGRGL...,DTLLTQSPDHLSVTLGERATISCRASQNIGTSLNWYQQKPGQSPKR...,NaN
4,humanization,74#human_0,EVQLVQSGAEVKKPGESLKISCQVSGYTFINYWIGWVRQMPGKGLE...,DIVMTQSPDSLTVSVGERATINCRSSQTLLNSNTQKNYVTWYQQKA...,NaN


In [6]:
# Not_successful_index

In [13]:
lab_human_df = humanized_df[humanized_df['type'] != 'mouse'].reset_index(drop=True)
lab_human_df

,type,name,order_name,h_seq,l_seq
0,humanized,h3A3-5,0_humanized,EVQLVESGGGLVQPGGSLRLSCAASGFTFSNFWMDWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCKASQDVSTDVAWYQQKPGKAPKL...
1,humanized,AB240,1_humanized,EVQLQESGPGLVKPSETLSLTCTVSGGSISDYGVNWIRQPPGKGLE...,DIQMTQSPSSLSASVGDRVTITCKASQAVSSAVAWYQQKPGKAPKL...
2,humanized,31HZ,2_humanized,QVQLVQSGAEVKKPGSSVKVSCKASGYTFTDYWMNWVRLAPGQGLE...,DVVITQSPDSLAVSLGERATINCKSSQSLLNTNSQKNYLAWYQQKP...
3,humanized,56HZ,3_humanized,QVQLQESGPGLVKPSQTLSLTCTVSGYFNSITSDYDWHWIRHHPGK...,DIVLTQSPATLSLSPGERATLSCRASQNIGTSIHWYQHKPGQSPRL...
4,humanized,74HZ,4_humanized,QVQLVQSGAEVKKPGSSVKVSCKASGYTFINYWMNWVRQAPGQGLE...,DIVITQSPDSLAVSLGERATINCKSSQTLLNSNTQKNYLAWYQQKP...
...,...,...,...,...,...
344,humanized,149G11-VH1VL1,350_humanized,QVQLVQSGAEVKKPGASVKVSCKASGYTFTNYWIHWVRQAPGQGLE...,DIVMTQSPDSLAVSLGERATINCKSSQNLLYNSNQKSYLAWYQQKP...
345,humanized,hz515H7 VH1 D76N VL2.1,352_humanized,EVQLVESGGGLVQPGRSLRLSCTASGFTFTDNYMSWVRQAPGKGLE...,DIVMTQSPDSLAVSLGERATMSCKSSQSLFNSRTRKNYLAWYQQKP...
346,humanized,hu16B5,353_humanized,QVQLVQSGAEVKKPGATVKISCKVSGFNIKDIYMHWVQQAPGKGLE...,DIVMTQSPDSLAVSLGERATINCKSSQSLLNSRTRKNYLAWYQQKP...
347,humanized,hu21B12,354_humanized,QVQLVQSGSELKKPGASVKVSCKASGYTFTNYGMHWVRQAPGQGLE...,DIVMTQSPDSLAVSLGERATINCKSSQSLLYSSNQKNYLAWYQQKP...


In [ ]:
lab_humab_df = pd.DataFrame(columns=['Raw_name', 'H-V-gene', 'H-Score', 'H-Threshold', 'H-Classification',
                                 'L-V-gene', 'L-Score', 'L-Threshold', 'L-Classification', 'H_seq', 'L_seq'])

def process_line(line):
    h_seq = line[1]['h_seq']
    l_seq = line[1]['l_seq']

    l_chain_type = Chain(l_seq, scheme='imgt').chain_type
    if l_chain_type == 'L':
        return True
        
    name = [line[1]['name']]
    job_name = line[1]['type'] + '_' + str(line[0])
    for retry in range(10):
        try:
            data = get_predict_result(job_name, h_seq, l_seq)
            if len(data) > 2:
                break
        except:
            time.sleep(5)
            continue
    if len(data) != 2:
        new_data = name + data
        new_line_df = pd.DataFrame([new_data], columns=lab_humab_df.columns)
        return new_line_df
    else:
        return None

# def main():
# ThreadPoolExecutor()

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_line, lab_human_df.iterrows()), total=len(lab_human_df)))

lab_humab_df = pd.concat([result for result in results if result is not None], ignore_index=True)
Not_successful_index = [i for i, result in enumerate(results) if result is None]



  0%|                                                                                                                           | 0/349 [00:00<?, ?it/s]

https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0555807
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0504009
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0564347
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0388449
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0689996
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0842942
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0744089
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0997406
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0817625
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0995492
[]
[]
[]
[]
[]
[]
[[['V-gene', 'Score', 'Threshold', 'Classification'], ['HV1', '0.005', '0.725', 'N

  0%|▎                                                                                                                | 1/349 [00:23<2:17:35, 23.72s/it]

[]
[]
[]
[]
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0881171
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0977100
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0595345
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0532114
[]
[]
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0679983
[]
[]
[]
[]
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0453830
[[['V-gene', 'Score', 'Threshold', 'Classification'], ['HV1', '0.325', '0.725', 'NOT HUMAN'], ['HV2', '0.000', '0.835', 'NOT HUMAN'], ['HV3', '0.115', '0.575', 'NOT HUMAN'], ['HV4', '0.000', '0.565', 'NOT HUMAN'], ['HV5', '0.045', '0.520', 'NOT HUMAN'], ['HV6', '0.000', '0.930', 'NOT HUMAN'], ['HV7', '0.675', '0.720', 'NOT HUMAN']], [['V-gene', 'Score', 'Threshold', 'Classification'], ['KV1', '0.820', '0.720', 'HUMAN'], ['KV2'

  1%|▋                                                                                                              | 2/349 [03:35<11:50:08, 122.79s/it]

...

https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0243145
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0434578
[]
[]
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0337693
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0610359
[]
[]
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0273034
[]
[]
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0110178
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0688382
[]
[]
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab
[]
[]
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0550157
[[['V-gene', 'Score', 'Threshold', 'Classification'], ['HV1', '0.465', '0.725', 'NOT HUMAN'], ['HV2', '0.000', '0.835', 'NOT HUMAN'], ['HV3', '0.080', '0.575', 'NOT HUMAN'], ['HV4', '0.

 71%|████████████████████████████████████████████████████████████████████████████████▌                                | 249/349 [31:04<15:06,  9.06s/it]

https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0313398
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0413158
[]
[]
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0154036
[]
[]
[[['V-gene', 'Score', 'Threshold', 'Classification'], ['HV1', '0.000', '0.725', 'NOT HUMAN'], ['HV2', '0.000', '0.835', 'NOT HUMAN'], ['HV3', '0.945', '0.575', 'HUMAN'], ['HV4', '0.000', '0.565', 'NOT HUMAN'], ['HV5', '0.000', '0.520', 'NOT HUMAN'], ['HV6', '0.000', '0.930', 'NOT HUMAN'], ['HV7', '0.000', '0.720', 'NOT HUMAN']], [['V-gene', 'Score', 'Threshold', 'Classification'], ['KV1', '0.965', '0.720', 'HUMAN'], ['KV2', '0.000', '0.720', 'NOT HUMAN'], ['KV3', '0.005', '0.720', 'NOT HUMAN'], ['KV4', '0.000', '0.655', 'NOT HUMAN'], ['KV5', '0.000', '0.815', 'NOT HUMAN'], ['KV6', '0.000', '0.475', 'NOT HUMAN']]]
['HV3', '0.945', '0.575', 'HUMAN', 'KV1', '0.965', '0.720', 'HUMAN']
[]
[]
https://opig.stats.ox.

 75%|████████████████████████████████████████████████████████████████████████████████████▌                            | 261/349 [32:13<10:46,  7.34s/it]

[[['V-gene', 'Score', 'Threshold', 'Classification'], ['HV1', '0.315', '0.725', 'NOT HUMAN'], ['HV2', '0.000', '0.835', 'NOT HUMAN'], ['HV3', '0.070', '0.575', 'NOT HUMAN'], ['HV4', '0.000', '0.565', 'NOT HUMAN'], ['HV5', '0.010', '0.520', 'NOT HUMAN'], ['HV6', '0.000', '0.930', 'NOT HUMAN'], ['HV7', '0.675', '0.720', 'NOT HUMAN']], [['V-gene', 'Score', 'Threshold', 'Classification'], ['KV1', '0.000', '0.720', 'NOT HUMAN'], ['KV2', '0.690', '0.720', 'NOT HUMAN'], ['KV3', '0.000', '0.720', 'NOT HUMAN'], ['KV4', '0.015', '0.655', 'NOT HUMAN'], ['KV5', '0.000', '0.815', 'NOT HUMAN'], ['KV6', '0.000', '0.475', 'NOT HUMAN']]]
[None, None, None, None, None, None, None, None]
[]
[]
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0970661
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0379815
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231208_0788407
https://opig.stats.ox.ac.uk/webapps/sabdab-sab

,Raw_name,H-V-gene,H-Score,H-Threshold,H-Classification,L-V-gene,L-Score,L-Threshold,L-Classification,H_seq,L_seq
0,MAK195human_0,HV3,0.975,0.575,HUMAN,KV1,0.730,0.720,HUMAN,QVQLVESGGGVVQPGGSLRLSCAASGFSLTDYGVAWVRQPPGKGLE...,DIQMAQSPSSLSASVGDRVTITCRASQAVSSALTWYQQIPGKSPKL...
1,74#human_0,HV5,1.000,0.520,HUMAN,KV4,0.950,0.655,HUMAN,EVQLVQSGAEVKKPGESLKISCQVSGYTFINYWIGWVRQMPGKGLE...,DIVMTQSPDSLTVSVGERATINCRSSQTLLNSNTQKNYVTWYQQKA...
2,11E6human_0,HV7,0.770,0.720,HUMAN,KV1,0.940,0.720,HUMAN,QVQLVQSESELKKPGASVRISCMASGYTFTNFGLNWVRQAPGQGFE...,AVQLFQSPSSVSASVGDRVTITCRASQNVGTAIAWYQQIKGKAPKP...
3,mouse anti-Tachuman_0,HV1,0.995,0.725,HUMAN,KV1,0.865,0.720,HUMAN,QVQLVQSGAEVKEPGASVEISCKASGYTFTSYRILWLRQVPGQGLE...,IQLTQSPSFLSASVGDTVIITCRASSSISYLAWYQQIPGKAPKLLI...
4,SC73.39human_0,HV1,0.995,0.725,HUMAN,KV4,0.985,0.655,HUMAN,QVQLVQSGAEVKKPGASVKVSCRTSGYSFTNYNFAWVRQAPRQGLE...,DIVMTQSPDSLAVSLGERATIKCKSSQSLLNSSNQKNYLAWYQQKP...
...,...,...,...,...,...,...,...,...,...,...,...
208,149G11human_0,HV5,0.985,0.520,HUMAN,KV4,0.995,0.655,HUMAN,EVRLVQSGAEVKKPGESLTISCKISGYTFTNYWIGWVRQMPGKGLE...,DIVMTQSPDSLAVSLGERATFNCKSSQNLLYNSNQKSYLAWYQQKP...
209,mu515H7human_0,HV3,0.960,0.575,HUMAN,KV4,0.980,0.655,HUMAN,EVQLVESGGGLVQPGGSLRLSCATSGFTFTDNYMDWVRQAPGKGLE...,IVLTQSPDSLAVSLGERATINCKSSQSLFNSRTRKNYLTWYQQKPG...
210,mu16B5human_0,HV1,0.845,0.725,HUMAN,KV4,0.965,0.655,HUMAN,QVRLVQSGAEVKKPGASVGVSCKASGFNIKDIYIHWVRQVPGQGPE...,DIVMTQFPNSLAVSLGERATINCKSSQSLLNSRTRKNYLSWYQKKA...
211,21B12human_0,HV7,0.940,0.720,HUMAN,KV4,1.000,0.655,HUMAN,QVQLVQSGSELKKPGASVKVSCKASGYTFTNYGLNWVRQAPGQGLE...,DIVMTQSPDSLAVSLGERATIDCKSSQSLLYSSNQKNYLTWYQQKP...


In [18]:
humab_df

,Raw_name,H-V-gene,H-Score,H-Threshold,H-Classification,L-V-gene,L-Score,L-Threshold,L-Classification,H_seq,L_seq
0,MAK195human_0,HV3,0.975,0.575,HUMAN,KV1,0.730,0.720,HUMAN,QVQLVESGGGVVQPGGSLRLSCAASGFSLTDYGVAWVRQPPGKGLE...,DIQMAQSPSSLSASVGDRVTITCRASQAVSSALTWYQQIPGKSPKL...
1,74#human_0,HV5,1.000,0.520,HUMAN,KV4,0.950,0.655,HUMAN,EVQLVQSGAEVKKPGESLKISCQVSGYTFINYWIGWVRQMPGKGLE...,DIVMTQSPDSLTVSVGERATINCRSSQTLLNSNTQKNYVTWYQQKA...
2,11E6human_0,HV7,0.770,0.720,HUMAN,KV1,0.940,0.720,HUMAN,QVQLVQSESELKKPGASVRISCMASGYTFTNFGLNWVRQAPGQGFE...,AVQLFQSPSSVSASVGDRVTITCRASQNVGTAIAWYQQIKGKAPKP...
3,mouse anti-Tachuman_0,HV1,0.995,0.725,HUMAN,KV1,0.865,0.720,HUMAN,QVQLVQSGAEVKEPGASVEISCKASGYTFTSYRILWLRQVPGQGLE...,IQLTQSPSFLSASVGDTVIITCRASSSISYLAWYQQIPGKAPKLLI...
4,SC73.39human_0,HV1,0.995,0.725,HUMAN,KV4,0.985,0.655,HUMAN,QVQLVQSGAEVKKPGASVKVSCRTSGYSFTNYNFAWVRQAPRQGLE...,DIVMTQSPDSLAVSLGERATIKCKSSQSLLNSSNQKNYLAWYQQKP...
...,...,...,...,...,...,...,...,...,...,...,...
208,149G11human_0,HV5,0.985,0.520,HUMAN,KV4,0.995,0.655,HUMAN,EVRLVQSGAEVKKPGESLTISCKISGYTFTNYWIGWVRQMPGKGLE...,DIVMTQSPDSLAVSLGERATFNCKSSQNLLYNSNQKSYLAWYQQKP...
209,mu515H7human_0,HV3,0.960,0.575,HUMAN,KV4,0.980,0.655,HUMAN,EVQLVESGGGLVQPGGSLRLSCATSGFTFTDNYMDWVRQAPGKGLE...,IVLTQSPDSLAVSLGERATINCKSSQSLFNSRTRKNYLTWYQQKPG...
210,mu16B5human_0,HV1,0.845,0.725,HUMAN,KV4,0.965,0.655,HUMAN,QVRLVQSGAEVKKPGASVGVSCKASGFNIKDIYIHWVRQVPGQGPE...,DIVMTQFPNSLAVSLGERATINCKSSQSLLNSRTRKNYLSWYQKKA...
211,21B12human_0,HV7,0.940,0.720,HUMAN,KV4,1.000,0.655,HUMAN,QVQLVQSGSELKKPGASVKVSCKASGYTFTNYGLNWVRQAPGQGLE...,DIVMTQSPDSLAVSLGERATIDCKSSQSLLYSSNQKNYLTWYQQKP...


In [19]:
human_h, human_l = humab_df['H-Classification'], humab_df['L-Classification']
both_h_l_index =  (human_h == 'HUMAN').astype(bool) & (human_l == 'HUMAN').astype(bool)
both_h_l_df = humab_df[both_h_l_index]
only_h_df = humab_df[human_h == 'HUMAN']
only_l_df = humab_df[human_l == 'HUMAN']

In [20]:
print('H humanization rate: {}'.format(len(only_h_df)/len(humab_df)))
print('L humanization rate: {}'.format(len(only_l_df)/len(humab_df)))
print('Both H and L humanization rate: {}'.format(len(both_h_l_df)/len(humab_df)))

H humanization rate: 0.9812206572769953
L humanization rate: 0.8826291079812206
Both H and L humanization rate: 0.8732394366197183


In [ ]:
# Define a new dataframe for saving results.
humab_df = pd.DataFrame(columns=['Raw_name', 'H-V-gene', 'H-Score', 'H-Threshold', 'H-Classification',
                                 'L-V-gene', 'L-Score', 'L-Threshold', 'L-Classification', 'H_seq', 'L_seq'])
# test_h = 'EVQLVESGGGLVQPGGSLRLSCAASGFTFSNFWMDWVRQAPGKGLEWIAGIRLKSYNYATHYAESVKGRFTISRDDSKSTVYLQMNSLRAEDTAVYYCTDWDGAYWGQGTLVTVSS'
# test_l = 'DIVMTQSPSSLSASVGDRVTITCKASQDVSTDVAWYQQKPGKSPKLLIYSASYRYTGVPSRFSGSGSGTDFTLTISSLQPEDIATYYCQQHYSTPFTFGQGTKLEIK'
Not_successful_index = []
for i, line in tqdm(enumerate(sample_human_df.iterrows())):
    h_seq = line[1]['hseq']
    l_seq = line[1]['lseq']
    name = [line[1]['name']]
    # print(name)
    job_name = line[1]['Specific'] + '_' + str(i)
    data = get_predict_result(job_name, h_seq, l_seq)
    # print(data)
    if data is not []:
        new_data = name + data
        new_line_df = pd.DataFrame([new_data], columns=humab_df.columns)
        humab_df = pd.concat([humab_df, new_line_df], ignore_index=True)
    else:
        Not_successful_index.append(i)
    
    

0it [00:00, ?it/s]

https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231124_0936481


1it [00:17, 17.81s/it]

[<table class="table table-results" style="text-align: center;">
<tr class="head"><th>V-gene</th><th>Score</th><th>Threshold</th><th>Classification</th></tr>
<tr>
<td>HV1</td>
<td>0.005</td>
<td>0.725</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
<tr>
<td>HV2</td>
<td>0.000</td>
<td>0.835</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
<tr>
<td>HV3</td>
<td>0.735</td>
<td>0.575</td>
<td style="background-color: #52d9a1;">HUMAN</td>
</tr>
<tr>
<td>HV4</td>
<td>0.000</td>
<td>0.565</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
<tr>
<td>HV5</td>
<td>0.000</td>
<td>0.520</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
<tr>
<td>HV6</td>
<td>0.000</td>
<td>0.930</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
<tr>
<td>HV7</td>
<td>0.000</td>
<td>0.720</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
</table>, <table class="table table-results" style="text-align: center;">
<tr class="head"

2it [00:35, 17.80s/it]

...

[<table class="table table-results" style="text-align: center;">
<tr class="head"><th>V-gene</th><th>Score</th><th>Threshold</th><th>Classification</th></tr>
<tr>
<td>HV1</td>
<td>1.000</td>
<td>0.725</td>
<td style="background-color: #52d9a1;">HUMAN</td>
</tr>
<tr>
<td>HV2</td>
<td>0.000</td>
<td>0.835</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
<tr>
<td>HV3</td>
<td>0.075</td>
<td>0.575</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
<tr>
<td>HV4</td>
<td>0.020</td>
<td>0.565</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
<tr>
<td>HV5</td>
<td>0.175</td>
<td>0.520</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
<tr>
<td>HV6</td>
<td>0.000</td>
<td>0.930</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
<tr>
<td>HV7</td>
<td>0.260</td>
<td>0.720</td>
<td style="background-color: #ed8282;">NOT HUMAN</td>
</tr>
</table>, <table class="table table-results" style="text-align: center;">
<tr class="head"

35it [10:52, 19.02s/it]

[]
[[['V-gene', 'Score', 'Threshold', 'Classification'], ['HV1', '0.005', '0.725', 'NOT HUMAN'], ['HV2', '0.000', '0.835', 'NOT HUMAN'], ['HV3', '0.815', '0.575', 'HUMAN'], ['HV4', '0.000', '0.565', 'NOT HUMAN'], ['HV5', '0.000', '0.520', 'NOT HUMAN'], ['HV6', '0.000', '0.930', 'NOT HUMAN'], ['HV7', '0.005', '0.720', 'NOT HUMAN']], [['V-gene', 'Score', 'Threshold', 'Classification'], ['KV1', '0.725', '0.720', 'HUMAN'], ['KV2', '0.000', '0.720', 'NOT HUMAN'], ['KV3', '0.040', '0.720', 'NOT HUMAN'], ['KV4', '0.040', '0.655', 'NOT HUMAN'], ['KV5', '0.010', '0.815', 'NOT HUMAN'], ['KV6', '0.000', '0.475', 'NOT HUMAN']]]
['HV3', '0.815', '0.575', 'HUMAN', 'KV1', '0.725', '0.720', 'HUMAN']
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231124_0253535


36it [11:10, 18.72s/it]

[]
[[['V-gene', 'Score', 'Threshold', 'Classification'], ['HV1', '0.005', '0.725', 'NOT HUMAN'], ['HV2', '0.000', '0.835', 'NOT HUMAN'], ['HV3', '0.815', '0.575', 'HUMAN'], ['HV4', '0.000', '0.565', 'NOT HUMAN'], ['HV5', '0.000', '0.520', 'NOT HUMAN'], ['HV6', '0.000', '0.930', 'NOT HUMAN'], ['HV7', '0.005', '0.720', 'NOT HUMAN']], [['V-gene', 'Score', 'Threshold', 'Classification'], ['KV1', '0.725', '0.720', 'HUMAN'], ['KV2', '0.000', '0.720', 'NOT HUMAN'], ['KV3', '0.040', '0.720', 'NOT HUMAN'], ['KV4', '0.040', '0.655', 'NOT HUMAN'], ['KV5', '0.010', '0.815', 'NOT HUMAN'], ['KV6', '0.000', '0.475', 'NOT HUMAN']]]
['HV3', '0.815', '0.575', 'HUMAN', 'KV1', '0.725', '0.720', 'HUMAN']
https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231124_0170044


In [94]:
sample_human_df

0             mouse
1      humanization
2             mouse
3      humanization
4             mouse
           ...     
627    humanization
628           mouse
629    humanization
630           mouse
631    humanization
Name: Specific, Length: 632, dtype: object

In [83]:
print(data)

None


In [28]:
response.url

'https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/humab_results/20231123_0990646'

In [29]:
result_test = requests.get(response.url)

In [31]:
result_test.text

'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="viewport" content="width=device-width, initial-scale=1">\n  \n  <!-- PAGE settings -->\n  <title>SAbPred: Hu-mAb Results\n</title>\n  <link href="/webapps/sabdab-sabpred/static/img/favicon.png" rel="shortcut icon" type="image/x-icon" />\n\n  <!-- CSS dependencies -->\n  <link rel="stylesheet" href="/webapps/sabdab-sabpred/static/css/wireframe.css">\n  <link rel="stylesheet" href="/webapps/sabdab-sabpred/static/css/google-fonts.css">\n  <!--<link href="https://fonts.googleapis.com/css?family=Comfortaa|Josefin+Slab:100,400|Source+Code+Pro" rel="stylesheet">-->\n  \n  <link href="/webapps/sabdab-sabpred/static/jsav/JSAVcustom.css" rel="stylesheet">\n  <link href="/webapps/sabdab-sabpred/static/Welcome_files/jquery-ui.css" rel="stylesheet">\n  <script src="/webapps/sabdab-sabpred/static/Welcome_files/jquery.js"></script> <!-- Must be loaded before JSAV -->\n  <script type="text/javascript" src="/webapps/sabdab-sabp

In [33]:
soup = BeautifulSoup(result_test.text, 'html.parser')

In [40]:
tables = soup.find_all('table', {'class': 'table table-results'})
all_table_data = []
for table in tables:
    table_data = []
    for row in table.find_all('tr'):
        row_data = []
        for cell in row.find_all(['th', 'td']):
            row_data.append(cell.text)
        table_data.append(row_data)
    all_table_data.append(table_data)
# soup

In [53]:
tables

[<table class="table table-results" style="text-align: center;">
 <tr class="head"><th>V-gene</th><th>Score</th><th>Threshold</th><th>Classification</th></tr>
 <tr>
 <td>HV1</td>
 <td>0.005</td>
 <td>0.725</td>
 <td style="background-color: #ed8282;">NOT HUMAN</td>
 </tr>
 <tr>
 <td>HV2</td>
 <td>0.000</td>
 <td>0.835</td>
 <td style="background-color: #ed8282;">NOT HUMAN</td>
 </tr>
 <tr>
 <td>HV3</td>
 <td>0.815</td>
 <td>0.575</td>
 <td style="background-color: #52d9a1;">HUMAN</td>
 </tr>
 <tr>
 <td>HV4</td>
 <td>0.000</td>
 <td>0.565</td>
 <td style="background-color: #ed8282;">NOT HUMAN</td>
 </tr>
 <tr>
 <td>HV5</td>
 <td>0.000</td>
 <td>0.520</td>
 <td style="background-color: #ed8282;">NOT HUMAN</td>
 </tr>
 <tr>
 <td>HV6</td>
 <td>0.000</td>
 <td>0.930</td>
 <td style="background-color: #ed8282;">NOT HUMAN</td>
 </tr>
 <tr>
 <td>HV7</td>
 <td>0.005</td>
 <td>0.720</td>
 <td style="background-color: #ed8282;">NOT HUMAN</td>
 </tr>
 </table>,
 <table class="table table-results" 

In [59]:
extract_human_data(all_table_data[:2])

['HV3', '0.815', '0.575', 'HUMAN', 'KV1', '0.725', '0.720', 'HUMAN']

In [43]:
extracted_data = []
for table_data in all_table_data[:2]:
    table_header = table_data[0]
    human_rows = [row for row in table_data if row[-1] == 'HUMAN']
    extracted_data.append((table_header, human_rows))

In [44]:
extracted_data

[(['V-gene', 'Score', 'Threshold', 'Classification'],
  [['HV3', '0.815', '0.575', 'HUMAN']]),
 (['V-gene', 'Score', 'Threshold', 'Classification'],
  [['KV1', '0.725', '0.720', 'HUMAN']])]

In [55]:
import pandas as pd

# 示例数据
data = [
    (['V-gene', 'Score', 'Threshold', 'Classification'], [['HV3', '0.815', '0.575', 'HUMAN']]),
    (['V-gene', 'Score', 'Threshold', 'Classification'], [['KV1', '0.725', '0.720', 'HUMAN']])
]

# 创建空的DataFrame
df = pd.DataFrame(columns=['H-V-gene', 'H-Score', 'H-Threshold', 'H-Classification',
                           'L-V-gene', 'L-Score', 'L-Threshold', 'L-Classification', 'Type'])

# 将数据添加到DataFrame中
H_test_data = data[0][1]
L_test_data = data[1][1]
    for row in rows:
        row_data = dict(zip(new_header, row))
        row_data['Type'] = new_header[0]
        new_row_df = pd.DataFrame(row_data, index=[0])
        df = pd.concat([df, new_row_df], ignore_index=True)

# 显示DataFrame
print(df)

  H-V-gene H-Score H-Threshold H-Classification L-V-gene L-Score L-Threshold  \
0      HV3   0.815       0.575            HUMAN      NaN     NaN         NaN   
1      NaN     NaN         NaN              NaN      KV1   0.725       0.720   

  L-Classification      Type  
0              NaN  H-V-gene  
1            HUMAN  L-V-gene  
